# Food Deserts in Davidson County

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import math
from sklearn.metrics.pairwise import haversine_distances # measure distance between lat, lng coordinates
import ast # used for converting strings to underlying datatypes
import requests
from bs4 import BeautifulSoup as BS
from shapely.ops import unary_union # used for merging shape objects
import folium

%matplotlib inline

**These are the criteria we will use for defining a food desert:**
1. Poverty level in a census tract is >= 20% (low income)
2. At least 33% of the census tract is >1 mile from a fresh food source in urban areas or >10 miles for rural areas (low access)

**For the analysis we will need:**
- The census tracts in Davidson County
- Poverty levels for each census tract
- Location of grocery stores

### Read in census tracts in Davidson County

In [ ]:
# cleaned from the TN census tracts found here: https://www2.census.gov/geo/tiger/GENZ2018/shp/
davidson_tracts = gpd.read_file('data/davidson_tracts.geojson')
davidson_tracts.head()

In [ ]:
davidson_tracts.boundary.plot(figsize = (20, 20));

Let's get the outline of Davidson county as well and set the Coordinate Reference System to match

In [ ]:
davidson_county = gpd.read_file('data/Davidson County Border (GIS).geojson')
davidson_county.crs = "EPSG:4326"
davidson_county.boundary.plot(figsize = (20, 20));

### Now that we have the census tract boundaries, we can look at poverty rates across them. For that we will import poverty count data from the census

In [ ]:
# cleaned from data found here: https://censusreporter.org/data/table/?table=B17001&geo_ids=14000US47037016000,05000US47037,04000US47,01000US,140|05000US47037&primary_geo_id=14000US47037016000
poverty_rates_davidson_tract = pd.read_csv('data/davidson_poverty_cleaned.csv', index_col = 0)
poverty_rates_davidson_tract.head()

Looking at the columns we can see that the data show the `Total` population for a tract, as well as the number above and below the poverty line in the past 12 months, both over all and broken down by sex and age range. We only need the total numbers, so to simplify we can remove the other columns. While we're at it we can drop the first 3 aggregation rows. Then we can use the number below the poverty line and the total population to determine the percent below the poverty level for each tract

In [ ]:
poverty_rates_davidson_tract = poverty_rates_davidson_tract.drop(columns = [c for c in poverty_rates_davidson_tract.columns if c not in ['geoid', 'name', 'Total:', 'Income in the past 12 months below poverty level:']])
poverty_rates_davidson_tract = poverty_rates_davidson_tract.drop([0, 1, 2])

poverty_rates_davidson_tract['pct_below_poverty_level'] = poverty_rates_davidson_tract['Income in the past 12 months below poverty level:']/poverty_rates_davidson_tract['Total:']
poverty_rates_davidson_tract.head()

### Now we have the poverty rates for each tract, we can determine the ones that are >20%. Since we will combine these data with additional geographic data, it would be best to first combine it with the census tract boundaries we already have

In [ ]:
poverty_rates_davidson_tract['NAME'] = poverty_rates_davidson_tract['name'].str.split(' ').str[2].str[:-1]
poverty_rates_davidson_tract.head()

In [ ]:
davidson_tracts = davidson_tracts.merge(poverty_rates_davidson_tract[['NAME', 'pct_below_poverty_level']],
                                        how = 'left',
                                        on = 'NAME')

# fill missing values with median for ease
davidson_tracts['pct_below_poverty_level'] = davidson_tracts['pct_below_poverty_level'].fillna(np.median(davidson_tracts['pct_below_poverty_level']))

Since we are interested in if a tract is above 20% poverty, we can add a new column that indicates if a tract is above or below 20%

In [ ]:
davidson_tracts['above_20_pct'] = (davidson_tracts['pct_below_poverty_level'] > 0.2).astype(int)
davidson_tracts.head()

_Now we have gotten to a good point where we can address the first part of our criteria for where food deserts could occur: **Which census tracts have >20% poverty?**_

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
ax = davidson_tracts.plot(ax=ax, column = 'above_20_pct', cmap='Set1', edgecolor = 'blue', alpha = 0.5)
ax = davidson_county.boundary.plot(ax=ax, color='blue');

The tracts in **gray** are above 20% poverty, while the ones in **red** are below 20% poverty

## Now we know which tracts have above 20% poverty, we can find stores that sell fresh produce

Finding all the grocery stores that can be accessed by Davidson County residents is a little tough. There are a few curated lists but they are behind a paywall and may be out of date. For this analysis we'll walk through an example of how the Google Maps API can be used to get Supermarkets and other stores.

Google Maps has many API endpoints, the one we'll use is the `nearbysearch` endpoint. For that we'll need to supply a **coordinate point**, a **radius**, and the **type** of store(s) to search for. The API does have limits so to make sure we are getting all the data we're expecting, we have to break the search down into smaller pieces. 

One important thing to consider is where to search for grocery stores. In general this seems obvious, but we should also consider the (literal) edge cases. For residents living at the edges of Davidson County, a grocery store could be within a mile _outside_ of the county limits. Additionally, the definition of a food desert is different for urban vs. rural areas. Urban areas look for grocery stores within _1_ mile, while rural areas look for grocery stores within _10_ miles. So some additional steps we need to take are:  
1. Determine which census tracts are considered urban vs. rural
2. If there are rural tracts, expand the search area to 10 miles around Davidson County
3. Divide the search area into smaller parts to ensure we capture all the stores

Determining "rural" census tracts is difficult, since urban areas are determined by population level, and census tracts are drawn to have approximately equal population levels. Luckily, Davidson County has outlined the **Nashville Urban Services District**, so let's start by comparing the urban area with the census tracts:

In [ ]:
davidson_service_districts = gpd.read_file('data/Service Districts (GIS).geojson')
davidson_service_districts.crs = "EPSG:4326"
davidson_service_districts

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
ax = davidson_tracts.plot(ax=ax, column = 'above_20_pct', cmap='Set1', edgecolor = 'blue', alpha = 0.5)
ax = davidson_county.boundary.plot(ax=ax, color='blue')
ax = davidson_service_districts[davidson_service_districts['name']=='Urban Services District'].plot(ax=ax, alpha = 0.4, edgecolor = 'black', linewidth = 3);

The urban area roughly overlaps with census tracts but not fully. For this analysis let's say if at least 50% of the tract is in the urban area, then it is an urban tract.

Determining how much of a shape overlaps with another is actually relatively straightforward. We can first subtract the urban area from a given tract, which will leave us with the tract area _outside_ of the urban area. Then we can divide that value by the total area of the tract. That will give us the proportion of the tract that is **rural**. We will use the `.area` attribute to work with numeric values. Once we have the ratio, we can determine `True` or `False`, is the ratio greater than 0.5. If it is, then the tract is **rural**

One good approach to determine if each tract is urban or rural is the `.apply()` method. This method will take a function, and _apply_ it to each row. There is no pre-built function to check if a tract is urban or rural so let's make our own!

In [ ]:
def check_urban_tract(row):
    return int(((row['geometry'] - urban_service_area).area/row['geometry'].area) > 0.5)

In [ ]:
urban_service_area = davidson_service_districts.loc[0, 'geometry']

davidson_tracts['is_rural'] = davidson_tracts.apply(check_urban_tract, axis = 1) # axis = 1 means apply to rows

In [ ]:
davidson_tracts.head()

Since there are rural tracts, we know we need to expand our search area, we can do that by creating a new GeoPandas object where a `buffer` is added around the county:

Geopandas buffers are calculated based on the crs of the gdf. Since we are using "EPSG:4326" we know the units are degrees. Using this conversion from the USGS (https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps?qt-news_science_products=0#qt-news_science_products): "One degree of latitude equals approximately 364,000 feet (69 miles), one minute equals 6,068 feet (1.15 miles), and one-second equals 101 feet." We can approximate the number of degrees in a mile as: 1/69 = 0.014492753623188406

In [ ]:
davidson_county_buffer_10 = gpd.GeoDataFrame(geometry = davidson_county['geometry'].buffer(0.14492753623188406),
                                          crs = "EPSG:4326")
ax = davidson_county_buffer_10.plot(figsize = (7, 7))
davidson_county.boundary.plot(ax = ax, edgecolor = 'white')

Now we can add a grid to the expanded search area to break the search down into smaller pieces. We can start by finding the absolute x and y boundaries of the search area:

In [ ]:
# Find total geographic bounds of search area

xmin,ymin,xmax,ymax = davidson_county_buffer_10.total_bounds

Based on a little experimentation, a good spacing for the grid points is by dividing the total bounds for Davidson County itself by 10. So let's do that now:

In [ ]:
# Find total geographic bounds of Davidson County

xmin_d,ymin_d,xmax_d,ymax_d = davidson_county.total_bounds

# Divide the length and width by 10 to get the increment between each grid point

x_increment = (xmax_d-xmin_d)/10
y_increment = (ymax_d-ymin_d)/10

Now we can proceed with setting up the search grid:

In [ ]:
# determine x coordinate values for grid points
grid_x_boundaries = [xmin]
new_bound = xmin
for i in range(int((xmax-xmin)/x_increment)+1):
    new_bound = new_bound + x_increment
    grid_x_boundaries.append(new_bound)
    
# determine x coordinate values for grid points
grid_y_boundaries = [ymin]
new_bound = ymin
for i in range(int((ymax-ymin)/y_increment)+1):
    new_bound = new_bound + y_increment
    grid_y_boundaries.append(new_bound)

In [ ]:
# get list of all lats and lons across all grid points
lons = []
lats = []
for left, right in zip(grid_x_boundaries[:-1], grid_x_boundaries[1:]):
    for top, bottom in zip(grid_y_boundaries[:-1], grid_y_boundaries[1:]):
        lats.append((top+bottom)/2)
        lons.append((left+right)/2)

By converting the coordinate pairs to `point` objects, we can put them into a geodataframe and then we can easily perform a number of geographic calculations.

In [ ]:
# Take each pair of longitude and latitude, combine them, and convert to point object
grid_points = gpd.points_from_xy(lons, lats)

In [ ]:
# put into geodataframe
grid_gdf = gpd.GeoDataFrame(geometry = grid_points, crs = "EPSG:4326")

In [ ]:
# Only keep points within the buffered Davidson county polygon
keep_points = []
for ind, p in grid_gdf['geometry'].iteritems():
    if p.within(davidson_county_buffer_10.loc[0, 'geometry']) or p.within(davidson_county_buffer_10.loc[1, 'geometry']):
        keep_points.append(ind)

grid_points_sub = grid_gdf.loc[keep_points].reset_index(drop=True)

In [ ]:
base = davidson_county_buffer_10.plot(color='white', edgecolor='black')

grid_points_sub.plot(ax=base, marker='o', color='red', markersize=5);

We can see that all the points are within the search area. The reason there is an additional circle in the search area is because Davidson County has a part that is disconnected from the main area.

Now that we have all the coordinate points we want to search nearby, we can figure out what an appropriate radius would be. The radius is measured in meters. Measuring distance with lat and lng is a little different since you need to take into consideration the curviture of the earth. To do that we can use the `haversine distance` formula.

In [ ]:
# function adapted from https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html
def dist_in_meters(point_1, point_2):
    point_1 = [math.radians(l) for l in [point_1.y, point_1.x]]
    point_2 = [math.radians(l) for l in [point_2.y, point_2.x]]
    dist_array_m = haversine_distances([point_1, point_2])*6371000
    return dist_array_m[0][1]

We will select a radius that is the distance between two adjacent points. This will cause some amount of overlap, but it will also maximize the chances a relevant store will be returned, if it is not retured from another point for some reason.

In [ ]:
grid_point_radius = dist_in_meters(grid_points_sub.loc[1, 'geometry'], grid_points_sub.loc[2, 'geometry'])

We can visualize the search area for each point on our map to make sure our entire search area is covered.

In [ ]:
grid_point_radius_mile = 3.0258050367212114828/69
grid_points_sub_buffers = gpd.GeoDataFrame(geometry = grid_points_sub['geometry'].buffer(grid_point_radius_mile),
                                          crs = "EPSG:4326")
f, ax = plt.subplots(1, figsize=(15, 15))
ax = davidson_tracts.boundary.plot(ax=ax, edgecolor = 'blue', color='blue', alpha = 0.15)
ax = davidson_county.boundary.plot(ax=ax, color='blue', alpha = 0.15)
ax = grid_points_sub_buffers.plot(ax=ax, color = '#ff7f00', alpha = 0.1)
grid_points_sub.plot(ax=ax, marker='o', color='#4e2853', markersize=5);

We can see that Davidson County is entirely covered, as well 10 miles surrounding.

### Now with grid points laid out across county we can figure out which types of stores to look for.

Google Maps has a number of business types (https://developers.google.com/places/supported_types). Looking through them, all relevant store types are these:

In [ ]:
all_types = ['bakery',
             'convenience_store',
             'department_store',
             'drugstore',
             'gas_station',
             'grocery_or_supermarket',
             'home_goods_store',
             'supermarket',
             'pharmacy']

To fully search for all stores across the entire county we would have to search each grid point for each store type. That will take a very long time. Additionally, it would require setting up an app on google to use for making the API requests. I've already run the code but we can walk through the steps for getting the data.

The function for actually making the API calls is here:

In [ ]:
# Modified from https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

def search_places_by_coordinate(location, radius, types, api_key, sleep_sec = 2):
    '''
    Send request to nearbysearch Google Maps endpoint
    
    location: The lat and lng to search nearby, "lat, lng"
    radius: The distance, in meters, to search around the location
    types: List of business types to search for
    api_key: Credentials provided by Google to authenticate use of the API
    sleep_sec: Number of seconds to wait between individual requests to throttle and avoid quotas
    '''
    # This is the endpoint where the request will be sent
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    places = [] # Where the responses will be saved
    
    # Formatting the request inputs
    params = {
        'location': location,
        'radius': radius,
        'types': types,
        'key': api_key
    }
    
    # Make the request to the endpoint with the associated parameters and save the output
    res = requests.get(endpoint_url, params = params)
    
    # Read the contents of the response, which is a json, into a dictionary to make it easier to work with
    results =  json.loads(res.content)
    
    # Add the results to the any previous results
    places.extend(results['results'])
    
    # Wait before the next request
    time.sleep(sleep_sec)
    
    # If there are still more items available, the response will contain a next_page_token to pick up at the same spot
    # As long as there is a next_page_token, keep making requests
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        places.extend(results['results'])
        time.sleep(sleep_sec)
    
    # Once there are no more next_page_tokens, return the full list of stores
    return places

Since many stores will be returned I created a separate list to add all the responses so they will persist even if there is an issue with a request or there is an interruption for some reason. This will allow me to pick up where it left off

In [ ]:
# creating output list in separate cell in case need to run for loop multiple times because of time out errors
responses = []

To systematically search the entire county for all store types I made a nested for loop that will search for each store type across all grid points and append the responses to the master list.

##### This cell will not run since no `api_key` is provided

In [ ]:
# This one can take a while, run with caution

for ind_2, t in enumerate(all_types):
    print(ind_2, t) # just to keep track of progress
    # if ind_2 >= 1: # uncomment and tab below over if need to start later in all_types list
    for ind, (lng, lat) in enumerate(list(zip(grid_points_sub['geometry'].x, grid_points_sub['geometry'].y))): # note that lat and lng are switched
        # print(ind, lat, lng) # again, to keep track of progress
        # if ind >= 0: # uncomment and tab below if need to start later in grid df
        location = '{}, {}'.format(lat, lng)
        responses.append(search_places_by_coordinate(location, grid_point_radius, t, api_key))

##### After this ran I cleaned the data, converted to a gdf, and saved it off as a .shp file. Now we can import it again and continue using the data.

In [ ]:
stores_gdf = gpd.read_file("data/google_api_stores_cleaned.shp")

In [ ]:
stores_gdf.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
ax.set_facecolor('#ebecfe')
ax = davidson_tracts.plot(ax=ax, column = 'above_20_pct', categorical = True, cmap = 'Set1', edgecolor = '#377eb8', alpha = 0.5)
ax = davidson_county.boundary.plot(ax=ax, color='#377eb8')
stores_gdf.plot(ax=ax, marker='o', color='#984ea3', markersize=5);

### Now that we've added the stores to the map we can start to see areas with many stores and areas with few stores, and how those compare to tracts above and below 20% poverty

However they may still be some cleaning we should do, as not all of the stores may actually offer fresh food. Looking at the first few rows of `stores_df`, we can see a **Twice Daily** and a **Panera Bread**, which aren't actually what we want. Let's try and filter down our stores to just ones that are classified as `supermarket`s:

In [ ]:
supermarkets = stores_gdf[stores_gdf['types'].apply(lambda x: 'supermarket' in x)]

In [ ]:
supermarkets.head()

In [ ]:
supermarkets.tail()

We can see that only keeping rows that have a `type` of `'supermarket'`, greatly reduces the number of stores. Also, in general this seems to be filtering how we want it to filter (we can see many **Publix** stores), but we can also see things that maybe shouldn't be there (like **Dollar General**). 

The influence of dollar stores on food deserts is an area if interest (https://www.cbsnews.com/news/dollar-stores-and-food-deserts-the-latest-struggle-between-main-street-and-corporate-america/). Though some sources indicate that they do not sell fresh produce, the Dollar General website indicates that produce is available in their stores (https://www.dollargeneral.com/catalogsearch/result/?q=produce) but there is no category for fresh foods (https://www.dollargeneral.com/food.html).

For this analysis we will exclude dollar stores from _stores selling fresh food_. That means we will need to remove them from our list.

In [ ]:
non_dollar_supermarkets = supermarkets[~supermarkets['name'].str.contains('Dollar')]

For completeness, let's double check the non-supermarkets to make sure we're not leaving anything we want behind

In [ ]:
non_supermarkets = stores_gdf[~stores_gdf['types'].apply(lambda x: 'supermarket' in x)]

In [ ]:
non_supermarkets['name'].value_counts()

A little investigation would show that a slight concern is **Kroger** is in the `non-supermarkets` list. Let's double check to see if **Kroger** is included in the `supermarkets` df.

In [ ]:
non_dollar_supermarkets[non_dollar_supermarkets['name'].str.contains('Kroger')]

There is one **Kroger**, but that means we are likely missing many others. Let's get the **Kroger** stores and add them back to the `supermarkets` list.

In [ ]:
kroger = stores_gdf[stores_gdf['name'].str.contains('Kroger')]

non_dollar_supermarkets = non_dollar_supermarkets.append(kroger)

non_dollar_supermarkets.tail()

With that cleaning done, let's plot the supermarkets on the map and see where we stand now:

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
ax.set_facecolor('#ebecfe')
ax = davidson_tracts.plot(ax=ax, column = 'above_20_pct', categorical = True, cmap = 'Set1', edgecolor = '#377eb8', alpha = 0.5)
ax = davidson_county.boundary.plot(ax=ax, color='#377eb8')
non_dollar_supermarkets.plot(ax=ax, marker='o', color='#984ea3', markersize=5);

There are far fewer now, but the spread seems similar.

## Now that we have a list of stores that sell fresh food, we can find the low income census tracts where at least a third is more than a mile (or 10 miles for rural tracts) away from them.

To address that we will add buffers around the stores that correspond to a 1 mile radius.

The buffers will be separate objects, so we will create a new gdf for them. Recall that we can approximate the number of degrees in a mile as: 1/69 = 0.014492753623188406

In [ ]:
urban_store_buffers = gpd.GeoDataFrame(geometry = non_dollar_supermarkets[non_dollar_supermarkets['rural_only']==0]['geometry'].buffer(0.014492753623188406),
                                          crs = "EPSG:4326")

rural_store_buffers = gpd.GeoDataFrame(geometry = non_dollar_supermarkets['geometry'].buffer(0.14492753623188406),
                                          crs = "EPSG:4326")

We will not add buffers for a 10 mile radius since those will obscure much of the map. 

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
ax.set_facecolor('#ebecfe')
ax = davidson_tracts.plot(ax=ax, column = 'above_20_pct', categorical = True, cmap = 'Set1', edgecolor = '#377eb8', alpha = 0.5)
ax = davidson_county.boundary.plot(ax=ax, color='#377eb8')
ax = non_dollar_supermarkets.plot(ax=ax, marker='o', color='#984ea3', markersize=10)
urban_store_buffers.plot(ax=ax, color = '#4daf4a', alpha = 0.1);

At this point we can start to see with more clarity where the food deserts actually are. To really nail it down, let's calculate the exact proportion of each census tract that is too far away from a fresh food source.

There are many ways we can accomplish this, but an easy way is to combine all the store buffers into a single shape, then check the proportion of each tract that overlaps with that shape. Since we are differentiating rural and urban tracts, let's also make different store buffers to check against.

To combine the buffer shapes, we can use the `unary_union()` function:

In [ ]:
urban_store_buffers = unary_union(urban_store_buffers['geometry'])
rural_store_buffers = unary_union(rural_store_buffers['geometry'])

In [ ]:
def check_low_food_access(row):
    if row['is_rural'] == 1:
        return (row['geometry'] - rural_store_buffers).area/row['geometry'].area
    else:
        return (row['geometry'] - urban_store_buffers).area/row['geometry'].area

In [ ]:
# check tract on row 3 of census tract df
check_low_food_access(davidson_tracts.loc[2, :])

The entire tract on row 3 of the census tract df is within a mile of a fresh food source, indicating it does not have low food access.

_**Now we can expand this to find all the possible food deserts in Davidson County.**_ Recall our criteria for possible food deserts:
1. Poverty level in a census tract is >= 20% (low income)
2. At least 33% of the census tract is >1 mile from a fresh food source in urban areas or >10 miles for rural areas (low access)

In [ ]:
davidson_tracts['ratio_low_food_access'] = davidson_tracts.apply(check_low_food_access, axis = 1)

davidson_tracts['possible_food_desert'] = ((davidson_tracts['ratio_low_food_access'] > 0.33) & (davidson_tracts['above_20_pct'] == 1)).astype(int)

In [ ]:
davidson_tracts.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
ax.set_facecolor('#ebecfe')
ax = davidson_tracts.plot(ax=ax, column = 'possible_food_desert', categorical = True, cmap = 'Set1', edgecolor = '#377eb8', alpha = 0.5)
ax = davidson_county.boundary.plot(ax=ax, color='#377eb8')
ax = non_dollar_supermarkets.plot(ax=ax, marker='o', color='#984ea3', markersize=10)
ax = davidson_service_districts[davidson_service_districts['name']=='Urban Services District'].plot(ax=ax, alpha = 0.2, edgecolor = 'black', linewidth = 3);

# According to our analysis, the census tracts in gray are possible food deserts. How would you interpret the results?

In [ ]:
nash_map = folium.Map(location =  [36.1612, -86.7775], zoom_start = 11)
folium.GeoJson(unary_union(davidson_tracts[davidson_tracts['possible_food_desert']==1]['geometry'])).add_to(nash_map)
nash_map